# Assess Base Model Endpoints using Azure AI Evaluation APIs


## Objective

This tutorial walks you through how to check prompts against different model endpoints on Azure AI Platform or other platforms.

We'll use a Python Class as the target application, which is sent to the Evaluate API from PromptFlow SDK to see how LLM models respond to the prompts.

We'll be using these Azure AI services: - [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

Allocate approximately 30 minutes to run this sample.


## About this example

This example illustrates how to evaluate model endpoint responses using azure-ai-evaluation.

 

## Before you begin

### Installation

Ensure you install the necessary packages to execute this notebook

In [ ]:
%pip install azure-ai-evaluation
%pip install promptflow-azure
%pip install promptflow-tracing
%pip install promptflow-evals

### Detailed Explanation of Parameters and Imports

In [ ]:
from pprint import pprint

import pandas as pd
import random
import os
from dotenv import load_dotenv
load_dotenv("../.credentials.env")

## App we're aiming for - Target Application

We will utilize the Evaluate API provided by the Prompt Flow SDK. This requires a target application or Python function to handle calls to LLMs and retrieve responses.

In the notebook, we will use an application target called `ModelEndpoints` to obtain answers from multiple model endpoints based on provided questions, also known as prompts.

This application target requires a list of model endpoints and their authentication keys. For simplicity, these have been provided in the `env_var` variable, which is passed into the init() function of `ModelEndpoints`.

In [ ]:
env_var = { 
    "gpt-35-turbo": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT35_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT35_API_KEY"),
    },
    "gpt-4": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT4_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT4_API_KEY"),
    },
    "gpt-4o": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT4o_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT4o_API_KEY"),
    },
   "gpt-4o-mini" : { 
        "endpoint" : os.environ.get("AZURE_OPENAI_GPT4o-mini_ENDPOINT"), 
        "key" : os.environ.get("AZURE_OPENAI_GPT4o-mini_API_KEY"), 
    },    
}


 Azure AI Project details to ensure traces and evaluation results are integrated into Azure AI Studio.

In [ ]:
# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

In [ ]:
print("Azure AI Project:",azure_ai_project)

## Model Endpoints
This code demonstrates how to call various model endpoints, configured based on the `env_var` set above. For any model in `env_var` that is not deployed in your AI project, please comment it out. If you wish to test a model not listed below, include its type in the `__call__` function and create a helper function to call the model's endpoint via REST.

## Data

The following code reads a JSON file named "ai_data.jsonl," which contains inputs for the Application Target function. Each line includes a question, context, and ground truth.

In [ ]:
df = pd.read_json("ai_data.jsonl", lines=True)
print(df.head())

## Configuration
To utilize the Relevance and Coherence Evaluator, we will use the Azure Open AI model details as a Judge, which can be included in the model configuration.

In [ ]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

## Execute the Evaluation

The following code executes the Evaluate API and utilizes the Content Safety, Relevance, and Coherence Evaluator to assess results from various models.

Below are the parameters required for the Evaluate API:

+   Data file (Prompts): This represents the data file 'ai_data.jsonl' in JSON format. Each line contains a question, context, and ground truth for evaluators.    
+   Application Target: This is the name of the Python class that can route calls to specific model endpoints using model names in conditional logic.  
+   Model Name: This is an identifier for the model, allowing custom code in the Application Target class to identify the model type and call the respective LLM model using the endpoint URL and authentication key.  
+   Evaluators: A list of evaluators provided to assess the given prompts (questions) as input and the output (answers) from LLM models.

In [ ]:
with open("target_ai_api/target_ai_api.py") as fin:
    print(fin.read())

In [ ]:
from target_ai_api.target_ai_api import ModelEndpoints

In [ ]:
import pathlib
import time
from target_ai_api.target_ai_api import ModelEndpoints
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)

relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)


models = ["gpt-35-turbo","gpt-4","gpt-4o","gpt-4o-mini"]

path = str(pathlib.Path(pathlib.Path.cwd())) + "/ai_data.jsonl"

for model in models:
    print(" Evaluating AI-assisted metrics - ", model)
    print("-----------------------------------")
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        azure_ai_project=azure_ai_project, 
        evaluation_name="Eval_" + model.title() + "_Run-" + str(randomNum),
        data=path,
        target=ModelEndpoints(env_var, model),
        
        evaluators={
            "relevance": relevance_evaluator,
            "groundedness": groundedness_evaluator,
            "coherence": coherence_evaluator,
            "fluency": fluency_evaluator,
            "similarity": similarity_evaluator,

        },
        evaluator_config={

            "relevance": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
                },

            "groundedness": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "coherence": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "fluency": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "similarity": {
                "column_mapping": {
                    "ground_truth": "${data.ground_truth}",
                    "context": "${data.context}", 
                    "query": "${data.query}",
                    "response": "${target.response}"}
            },
        },
    )
    #time.sleep(60) ## To avoid rate limiting throttling


View the results

In [ ]:
pd.DataFrame(results["rows"])